In [19]:
import pandas
import numpy as np
import pyproj
import math

In [65]:
df = pandas.read_parquet("./emission_corrected.parquet")
df.head()

# NOTE
# x and y are swapped in the file
# y is latitude (hence x)

,id,NOx,electricity,type,x,y,time
0,truck0,38.95,0.00,truck_petrol,55.708206,12.572871,0.0
1,veh0,0.00,1.03,veh_hybrid,55.693703,12.576520,0.0
2,truck0,41.67,0.00,truck_petrol,55.708209,12.572886,1.0
3,veh0,0.00,0.00,veh_hybrid,55.693773,12.576334,1.0
4,truck0,42.54,0.00,truck_petrol,55.708214,12.572912,2.0


In [70]:
df['type'].unique()

array(['truck_petrol', 'veh_hybrid', 'veh_diesel', 'veh_ev',
       'truck_hybrid', 'veh_petrol'], dtype=object)

In [63]:
print("Fixing coordinates swapped")
for i, vehicle in df.iterrows():
    tmp = df.at[i, 'x']
    df.at[i, 'x'] = df.at[i, 'y']
    df.at[i, 'y'] = tmp
print("Done")
df.to_parquet('./emission_corrected.parquet')

Fixing coordinates swapped
Done


In [40]:
timeSteps = np.max(df['time'].unique())
print(timeSteps)

4021.0


In [24]:
def coordinate_deg_to_meters(lon, lat):
    y = lat * 111320.0 #1 degree = 111.32km
    x = lon * 40075000.0 * math.cos(lat * math.pi / 180.0) / (2 * math.pi)
    return (x, y)

In [30]:
firstVehicle = df[df['time'] == 0].loc[0]
print(df.loc)
print(coordinate_deg_to_meters(firstVehicle.x, firstVehicle.y))

(45180488.46643454, 6201437.49192)


In [35]:
print("Converting coordinates from degrees to meters...")
for i, vehicle in df.iterrows():
    x, y = coordinate_deg_to_meters(vehicle.x, vehicle.y)
    df.at[i, 'x'] = x
    df.at[i, 'y'] = y
print("Done")

Converting coordinates from degrees to meters...
Done


In [66]:
#df.head()
print(df[df['id'] == 'truck0'])

           id    NOx  electricity          type          x          y   time
0      truck0  38.95          0.0  truck_petrol  55.708206  12.572871    0.0
2      truck0  41.67          0.0  truck_petrol  55.708209  12.572886    1.0
4      truck0  42.54          0.0  truck_petrol  55.708214  12.572912    2.0
6      truck0  48.50          0.0  truck_petrol  55.708223  12.572956    3.0
9      truck0  49.86          0.0  truck_petrol  55.708234  12.573015    4.0
...       ...    ...          ...           ...        ...        ...    ...
17880  truck0  41.47          0.0  truck_petrol  55.713082  12.579873  236.0
18016  truck0  41.76          0.0  truck_petrol  55.713079  12.579791  237.0
18152  truck0  46.58          0.0  truck_petrol  55.713078  12.579703  238.0
18287  truck0  40.61          0.0  truck_petrol  55.713076  12.579616  239.0
18423  truck0   0.00          0.0  truck_petrol  55.713075  12.579533  240.0

[241 rows x 7 columns]


In [69]:
# Projection
min_x = df['x'].min()
max_x = df['x'].max()
min_y = df['y'].min()
max_y = df['y'].max()
resolution = max(max_x - min_x, max_y - min_y)
mapWidth = 40
mapHeight = 40
cellSize = 17
padding_top = (1.0 - ((max_y - min_y) / resolution)) * mapHeight * cellSize / 2
padding_left = (1.0 - ((max_x - min_x) / resolution)) * mapWidth * cellSize / 2
print(padding_top)
print(padding_left)

# Mapping first vehicle
print(padding_left + ((df.loc[0].x - min_x) / resolution)* mapWidth * cellSize)
print(padding_top + ((df.loc[0].y - min_y) / resolution) * mapHeight * cellSize)

0.0
81.88180671788642
432.8734380373308
286.40605296341533
